# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.108954e+02     1.609858e+00
 * time: 0.07342696189880371
     1     1.075520e+01     9.229743e-01
 * time: 1.3655188083648682
     2    -1.190452e+01     1.027031e+00
 * time: 1.4644010066986084
     3    -3.420671e+01     7.409714e-01
 * time: 1.6332218647003174
     4    -4.772707e+01     5.543102e-01
 * time: 1.751297950744629
     5    -5.711536e+01     2.471401e-01
 * time: 1.8831830024719238
     6    -5.995878e+01     1.728113e-01
 * time: 1.9832370281219482
     7    -6.097852e+01     5.205887e-02
 * time: 2.092484951019287
     8    -6.134704e+01     9.773166e-02
 * time: 2.1825509071350098
     9    -6.160993e+01     5.521997e-02
 * time: 2.2744410037994385
    10    -6.179592e+01     2.686380e-02
 * time: 2.366421937942505
    11    -6.195527e+01     2.035834e-02
 * time: 2.458930015563965
    12    -6.202498e+01     1.844130e-02
 * time: 2.5653939247131348
    13    -6.209441e+01     1.493593e-02
 * time: 2.6568078994750

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557680
    AtomicNonlocal      14.8522642
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336822

    total               -62.261666462778
